In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Loading modules

In [2]:
import mbuild as mb
import foyer
from foyer import Forcefield

import mbuild.formats.charmm_writer as mf_charmm

import mosdef_cp2k_writer


import parmed as pmd
from constrainmol import ConstrainedMolecule
import unyt as u

from mosdef_cp2kmcpy.mc import MC
import setter
warnings.filterwarnings("ignore", category=DeprecationWarning)
 

## The forcefield file

In [3]:
!cat dinitrogen_ff.xml

<ForceField>
    <AtomTypes>
        <Type name="N" def="[N;X1](N)"
              class="N2" element="N" mass=" 14.0067" desc="N in N2"
              doi="random" />
    </AtomTypes>
    <HarmonicBondForce>
        <Bond class1="N2" class2="N2" length="0.1121" k="650500"/>
    </HarmonicBondForce>
    <HarmonicAngleForce>
    </HarmonicAngleForce>
    <RBTorsionForce>
    </RBTorsionForce>
    <NonbondedForce coulomb14scale="0" lj14scale="0">
        <Atom type="N" charge="0" sigma="0.32973" epsilon="0.30491436"/>
    </NonbondedForce>
</ForceField>

## Defining the molecule



In [4]:

nitrogen_res_name = 'N2'
FF_file_nitrogen = 'dinitrogen_ff.xml'
nitrogen_FF = Forcefield(forcefield_files=FF_file_nitrogen)
nitrogen_mb = mb.load('N#N', smiles=True)

nitrogen_mb.name = nitrogen_res_name

nitrogen_typed=nitrogen_FF.apply(nitrogen_mb)



constrain_mol = ConstrainedMolecule(nitrogen_typed)
constrain_mol.solve()
nitrogen_optimized_mb=mb.clone(nitrogen_mb)
nitrogen_optimized_mb.xyz=constrain_mol.xyz/10
print(nitrogen_optimized_mb.xyz)

/home/rs/anaconda3/envs/mosdef_cp2k/lib/python3.8/site-packages/foyer/forcefield.py:448: UserWarning: No force field version number found in force field XML file.
  warnings.warn(
/home/rs/anaconda3/envs/mosdef_cp2k/lib/python3.8/site-packages/foyer/forcefield.py:460: UserWarning: No force field name found in force field XML file.
  warnings.warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:687: UserWarning: No unitcell detected for pybel.Molecule N#N	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))
/home/rs/anaconda3/envs/mosdef_cp2k/lib/python3.8/site-packages/parmed/openmm/topsystem.py:240: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  warnings.warn('Adding what seems to be Urey-Bradley terms before ' # pragma: no cover


[[ 0.09788612 -0.00626626 -0.00349077]
 [ 0.20998612 -0.00626626 -0.00349077]]


## Creating .psf (Topology) and .inp (CHARMM potential file) using mosdef_charmm_writer


In [5]:
#random box to be used in mosdef_charmm_writer
nitrogen_box_bias  = mb.fill_box(compound=[nitrogen_optimized_mb ],
                             n_compounds=[1] ,
                            box=[1.0, 1.0, 1.0] )

### mosdef_charmm_writer

In [6]:
FF_Dict = {nitrogen_mb.name:FF_file_nitrogen }

residues_List = [nitrogen_mb.name ]




nitrogen_box_bias.save("nitrogen_bias_coord.xyz",overwrite=True)

print('Running: GOMC FF file, and the psf and pdb files for the biasing potential file')
mf_charmm.charmm_psf_psb_FF(nitrogen_box_bias ,
                            'nitrogen_bias',
                            FF_filename ="nitrogen_charmm_bias" ,
                            forcefield_selection = FF_Dict,
                            residues= residues_List ,
                            bead_to_atom_name_dict = {},
                            fix_residue = None,
                            reorder_res_in_pdb_psf = False
                            )

! rm *.pdb *.xyz

Running: GOMC FF file, and the psf and pdb files for the biasing potential file
write_gomcdata: forcefield_selection = {'N2': 'dinitrogen_ff.xml'}, residues = ['N2']
******************************

GOMC FF writing each residues FF as a group for structure_0
forcefield type from compound = {'N2': 'dinitrogen_ff.xml'}
coulomb14scale from compound = {'N2': 0.0}
lj14scale from compound = {'N2': 0.0}
unique_types = ['N_N2']
No urey bradley terms detected, will use angle_style harmonic
******************************

writing the GOMC force field file 
NBFIX_Mixing not used or no mixing used for the non-bonded potentials out
forcefield_dict = {0: 'N_N2'}
******************************

write_charmm_psf file is running
write_charmm_psf: forcefield_selection = {'N2': 'dinitrogen_ff.xml'}, residues = ['N2']
******************************

No urey bradley terms detected
******************************

write_charmm_pdb file is running
write_charmm_pdb: residues == ['N2']
fix_residue = None
fix_res

/home/rs/anaconda3/envs/mosdef_cp2k/lib/python3.8/site-packages/parmed/openmm/topsystem.py:240: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  warnings.warn('Adding what seems to be Urey-Bradley terms before ' # pragma: no cover


### Now we have the .psf (topology) file and .inp (charmm potential) file

## Creating CP2K MC simulation

In [7]:
molecule_list=[nitrogen_optimized_mb]
box=mb.box.Box(lengths=[1,1,1])
#Comment why pressure is needed
q=MC(molecule_list=molecule_list,n_box=2,n_molecules_each_box=[[49],[1]], box_list=[box,box],cutoff=200,functional='PBE',
     basis_set={'N':'DZVP-MOLOPT-GTH'}, periodicity=['XYZ']*2,ensemble='GEMC_NVT',seed=1,project_name='N2_NVT_GEMC',restart='FALSE',pressure=1*u.bar)

In [8]:
q.mc_initialization()

scf_tolerance not specified, set as 1e-6
basis_set_filename not defined, set as BASIS_MOLOPT
potential_filename not specified, set as GTH_POTENTIALS
n_steps not specified, set as 1000
n_ff_moves not specified, set as 8
nswapmoves not specified, set as 640, will be ignore if n_box<2
output trajectory format set as XYZ
input_filename not specified, set as ['GEMC_NVT_box1.inp', 'GEMC_NVT_box2.inp']
output_filename not specified, set as N2_NVT_GEMC_mc_output.out
temperature not defined, set as 298 K
You can change default settings in setter.mc_files


In [9]:

q.topology_filename=['nitrogen_bias.psf']

# move_probabilities=[pmavbmc,pmcltrans,pmhmc,pmswap,pmtraion,pmtrans,pmvolume]
#volume moves = PMVOLUME
# swap moves = PMSWAP - PMVOLUME
# AVBMC moves = PMAVBMC - PMSWAP - PMVOLUME
# an “inner” move = 1.0 - (PMAVBMC + PMSWAP + PMVOLUME)
#conformational changes = “inner” move percentage × PMTRAION
# molecular translation = “inner” move percentage × (PMTRANS - PMTRAION)
# molecular rotation = “inner” move percentage × (1.0 - PMTRANS - PMTRAION)

q.move_probabilities=[0,0.0,0,0.4,0.5,0.75,0.05]



# mol_probabilities=[[PMAVBMC_MOL,PMSWAP_MOL , PMTRAION_MOL, PMTRANS_MOL,PMROT_MOL],[]]
q.mol_probabilities=[[[1],[1],[1],[1],[1]],[[1],[1],[1],[1],[1]]]



#avbmc probabilities=[[AVBMC_ATOM,AVBMC_RMIN,AVBMC_RMAX,PBIAS],[]]
q.avbmc_probabilities=[[[1],[1],[1],[1]],[[1],[1],[1],[1]]]

q.charmm_potential_file='nitrogen_charmm_bias.inp'

In [10]:
setter.mc_files(q)

MC initial structure saved as ['N2_NVT_GEMC_box1_initial.xyz', 'N2_NVT_GEMC_box2_initial.xyz']
MC input file saved as GEMC_NVT_box1.inp
MC input file saved as GEMC_NVT_box2.inp


In [11]:
!ls

GEMC_NVT_box1.inp	      bias_template.inp  nitrogen_charmm_bias.inp
GEMC_NVT_box2.inp	      changeLog.out	 old.ipynb
N2_NVT_GEMC_box1_initial.xyz  dinitrogen_ff.xml  setter.py
N2_NVT_GEMC_box2_initial.xyz  log.txt		 trajectories
Untitled.ipynb		      md-changeLog.out
__pycache__		      nitrogen_bias.psf


In [2]:
!cp2k.popt -i GEMC_NVT_box1.inp -i GEMC_NVT_box2.inp

 DBCSR| Multiplication driver                                               BLAS
 DBCSR| Multrec recursion limit                                              512
 DBCSR| Multiplication stack size                                           1000
 DBCSR| Maximum elements for images                                    UNLIMITED
 DBCSR| Multiplicative factor virtual images                                   1
 DBCSR| Multiplication size stacks                                             3
 DBCSR| Number of 3D layers                                               SINGLE
 DBCSR| Use MPI memory allocation                                              T
 DBCSR| Use RMA algorithm                                                      F
 DBCSR| Use Communication thread                                               T
 DBCSR| Communication thread load                                             87


  **** **** ******  **  PROGRAM STARTED AT               2022-09-27 14:01:17.001
 ***** ** ***  *** **   PR